# Semantic Search on Computer Science Research Papers abstract using Sentence Transformers & FAISS

Utilizing Sentence Transformers to perform semantic search on research paper abstracts. Finding similarity using cosine similarity metric between the query vector and each abstract vector. Employing FAISS for demonstrating how to scale search efficiently for large datasets.

In [1]:
!pip install sentence_transformers
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=00088d83902ed5bf070d105e8ce0386fe7ebd5a2c83ef3edee55b25f5590ec4f
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 57.2 MB/s eta 0:00:0000:0100:01


In [2]:
import numpy as np
import pandas as pd

from string import digits
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from tqdm import tqdm, notebook

from sentence_transformers import SentenceTransformer, util
import torch

import faiss

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/kaggle/input/arxiv-cs-papers-abstract-from-2010/cs_arxiv_from_2010.csv


In [3]:
docs_df = pd.read_csv('/kaggle/input/arxiv-cs-papers-abstract-from-2010/cs_arxiv_from_2010.csv')
docs_df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,authors,title,category,abstract
0,704.0213,Ketan D. Mulmuley Hariharan Narayanan,Geometric Complexity Theory V: On deciding non...,['cs.CC'],This article has been withdrawn because it h...
1,704.1409,Yao HengShuai,Preconditioned Temporal Difference Learning,"['cs.LG', 'cs.AI']",This paper has been withdrawn by the author....
2,704.1829,"Stefan Felsner, Kamil Kloch, Grzegorz Matecki,...",On-line Chain Partitions of Up-growing Semi-or...,['cs.DM'],On-line chain partition is a two-player game...
3,705.0561,Jing-Chao Chen,Iterative Rounding for the Closest String Problem,"['cs.DS', 'cs.CC']",The closest string problem is an NP-hard pro...
4,705.1025,David Eppstein,Recognizing Partial Cubes in Quadratic Time,['cs.DS'],We show how to test whether a graph with n v...


In [4]:
device = 'cuda'
if torch.cuda.is_available():      
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


#### Concatenating title and abstract for the search target

In [7]:
docs_text = (docs_df['title'] + ' ' + docs_df['abstract']).values.tolist()
docs_text[:5]

["Geometric Complexity Theory V: On deciding nonvanishing of a generalized\n  Littlewood-Richardson coefficient   This article has been withdrawn because it has been merged with the earlier\narticle GCT3 (arXiv: CS/0501076 [cs.CC]) in the series. The merged article is\nnow available as:\n  Geometric Complexity Theory III: on deciding nonvanishing of a\nLittlewood-Richardson Coefficient, Journal of Algebraic Combinatorics, vol. 36,\nissue 1, 2012, pp. 103-110. (Authors: Ketan Mulmuley, Hari Narayanan and Milind\nSohoni)\n  The new article in this GCT5 slot in the series is:\n  Geometric Complexity Theory V: Equivalence between blackbox derandomization\nof polynomial identity testing and derandomization of Noether's Normalization\nLemma, in the Proceedings of FOCS 2012 (abstract), arXiv:1209.5993 [cs.CC]\n(full version) (Author: Ketan Mulmuley)\n",
 'Preconditioned Temporal Difference Learning   This paper has been withdrawn by the author. This draft is withdrawn for its\npoor quality in

## Pre-processing

In [8]:
def clean_text(text, remove_stopwords=True):
    text = text.lower()
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words('english'))
        text = [w for w in text if w not in stops]
        text = ' '.join(text)
        
    return text

In [9]:
def clean_data(data):
    cleaned_data = []
    for doc in notebook.tqdm(data):
        text = clean_text(doc, False)
        cleaned_data.append(text)
    return cleaned_data

In [10]:
cleaned_docs = clean_data(docs_text)

  0%|          | 0/484027 [00:00<?, ?it/s]

#### Loading the transformer model
all-MiniLM-L6-v2 has given really good results on semantic search tasks

In [11]:
embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)

In [14]:
def get_embeddings(data):
    return embedder.encode(data, convert_to_tensor=True)

In [15]:
abstract_embeddings = get_embeddings(cleaned_docs)

Batches:   0%|          | 0/15126 [00:00<?, ?it/s]

In [16]:
query = ['temporal expression extraction']
query_embedding = get_embeddings(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

#### Computing cosine similarity metric  for query embedding against each abstract embedding

In [17]:
%%time

cos_scores = util.cos_sim(query_embedding, abstract_embeddings)
cos_scores.shape

CPU times: user 553 µs, sys: 3 ms, total: 3.55 ms
Wall time: 4.23 ms


torch.Size([1, 484027])

In [18]:
cos_scores[0]

tensor([ 0.0448,  0.3572,  0.0271,  ...,  0.1939, -0.0275,  0.0822],
       device='cuda:0')

In [19]:
top_results = torch.topk(cos_scores[0], k=5)
top_results

torch.return_types.topk(
values=tensor([0.7779, 0.6950, 0.6800, 0.6714, 0.6708], device='cuda:0'),
indices=tensor([ 16069, 154036,  66896,  13600, 192765], device='cuda:0'))

Closest match

In [20]:
docs_df.iloc[16069].abstract

'  Automatic annotation of temporal expressions is a research challenge of great\ninterest in the field of information extraction. In this report, I describe a\nnovel rule-based architecture, built on top of a pre-existing system, which is\nable to normalise temporal expressions detected in English texts. Gold standard\ntemporally-annotated resources are limited in size and this makes research\ndifficult. The proposed system outperforms the state-of-the-art systems with\nrespect to TempEval-2 Shared Task (value attribute) and achieves substantially\nbetter results with respect to the pre-existing system on top of which it has\nbeen developed. I will also introduce a new free corpus consisting of 2822\nunique annotated temporal expressions. Both the corpus and the system are\nfreely available on-line.\n'

In [21]:
docs_df.iloc[154036].abstract

'  The current leading paradigm for temporal information extraction from text\nconsists of three phases: (1) recognition of events and temporal expressions,\n(2) recognition of temporal relations among them, and (3) time-line\nconstruction from the temporal relations. In contrast to the first two phases,\nthe last phase, time-line construction, received little attention and is the\nfocus of this work. In this paper, we propose a new method to construct a\nlinear time-line from a set of (extracted) temporal relations. But more\nimportantly, we propose a novel paradigm in which we directly predict start and\nend-points for events from the text, constituting a time-line without going\nthrough the intermediate step of prediction of temporal relations as in earlier\nwork. Within this paradigm, we propose two models that predict in linear\ncomplexity, and a new training loss using TimeML-style annotations, yielding\npromising results.\n'

In [22]:
abstract_embeddings.shape

torch.Size([484027, 384])

In [23]:
query_embedding.shape

torch.Size([1, 384])

## Experimenting with FAISS
Utilizing FAISS for creating index for the abstract vectors. It can be used to perform Apporximate Nearest Neighbor Search post clustering of the vectors.

In [24]:
abstract_embeddings_np = abstract_embeddings.detach().cpu().numpy()
query_embedding_np = query_embedding.detach().cpu().numpy()

Data is less, a flat index can be used.

In [25]:
dim=384

In [26]:
index = faiss.IndexFlatL2(dim)
index.add(abstract_embeddings_np)

In [27]:
%%time

dist, top_k = index.search(query_embedding_np, 5)
top_k

CPU times: user 69.7 ms, sys: 0 ns, total: 69.7 ms
Wall time: 68.3 ms


array([[ 16069, 154036,  66896,  13600, 192765]])

In [28]:
dist

array([[0.444108  , 0.6100239 , 0.6400416 , 0.65727806, 0.6584898 ]],
      dtype=float32)

tensor([ 16069, 154036,  66896,  13600, 192765], device='cuda:0')
faiss internally uses the same scoring metric hence returns the same top 5 results.

In [29]:
index.is_trained

True

In [30]:
index.ntotal

484027

#### Experimenting with different index which involves training and we won't be using a flat index for this part.

Adding Partitioning to the Index. Similar to Elasticsearch partitioning on index concept.

In [31]:
nlist = 200  # how many cells
quantizer = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFFlat(quantizer, dim, nlist)

In [32]:
index.is_trained

False

In the training process clusters are formed based on the vector data and on adding the vectors to the index, they are assigned to different clusters.
The index can stored to a disk and retrieved later for search.

In [35]:
%%time

index.train(abstract_embeddings_np)
index.is_trained

CPU times: user 69 µs, sys: 0 ns, total: 69 µs
Wall time: 75.3 µs


True

In [36]:
index.ntotal

0

Adding embeddings to the trained index

In [37]:
%%time

index.add(abstract_embeddings_np)

CPU times: user 1.25 s, sys: 468 ms, total: 1.71 s
Wall time: 1.72 s


In [38]:
index.ntotal

484027

In [39]:
%%time

dist, top_k = index.search(query_embedding_np, 5)  # search
top_k

CPU times: user 1.41 ms, sys: 4 µs, total: 1.41 ms
Wall time: 1.03 ms


array([[ 16069, 347324, 368076,  28773,  13593]])

basically the search is taking less than a ms compared to 73 ms in the previous search without partitioning but the results are less accurate. Let's see some of the examples

In [40]:
docs_df.iloc[16069].abstract

'  Automatic annotation of temporal expressions is a research challenge of great\ninterest in the field of information extraction. In this report, I describe a\nnovel rule-based architecture, built on top of a pre-existing system, which is\nable to normalise temporal expressions detected in English texts. Gold standard\ntemporally-annotated resources are limited in size and this makes research\ndifficult. The proposed system outperforms the state-of-the-art systems with\nrespect to TempEval-2 Shared Task (value attribute) and achieves substantially\nbetter results with respect to the pre-existing system on top of which it has\nbeen developed. I will also introduce a new free corpus consisting of 2822\nunique annotated temporal expressions. Both the corpus and the system are\nfreely available on-line.\n'

first one is still the same

In [41]:
docs_df.iloc[347324].abstract

'  The understanding of time expressions includes two sub-tasks: recognition and\nnormalization. In recent years, significant progress has been made in the\nrecognition of time expressions while research on normalization has lagged\nbehind. Existing SOTA normalization methods highly rely on rules or grammars\ndesigned by experts, which limits their performance on emerging corpora, such\nas social media texts. In this paper, we model time expression normalization as\na sequence of operations to construct the normalized temporal value, and we\npresent a novel method called ARTime, which can automatically generate\nnormalization rules from training data without expert interventions.\nSpecifically, ARTime automatically captures possible operation sequences from\nannotated data and generates normalization rules on time expressions with\ncommon surface forms. The experimental results show that ARTime can\nsignificantly surpass SOTA methods on the Tweets benchmark, and achieves\ncompetitive r

As we can see the second abstract doesn't exactly match the query - temporal expression extraction but is close to it. While the previous result without partiotioing the second abstract was a very close match for the query.

Third result

In [42]:
docs_df.iloc[368076].abstract

'  Extracting temporal relations among events from unstructured text has\nextensive applications, such as temporal reasoning and question answering.\nWhile it is difficult, recent development of Neural-symbolic methods has shown\npromising results on solving similar tasks. Current temporal relation\nextraction methods usually suffer from limited expressivity and inconsistent\nrelation inference. For example, in TimeML annotations, the concept of\nintersection is absent. Additionally, current methods do not guarantee the\nconsistency among the predicted annotations. In this work, we propose SMARTER,\na neural semantic parser, to extract temporal information in text effectively.\nSMARTER parses natural language to an executable logical form representation,\nbased on a custom typed lambda calculus. In the training phase, dynamic\nprogramming on denotations (DPD) technique is used to provide weak supervision\non logical forms. In the inference phase, SMARTER generates a temporal relation\n

We can imporve the search accuracy by increasing the nprobe value which increases the number of cells searched.

In [43]:
index.nprobe

1

In [44]:
index.nprobe = 10
index.nprobe

10

In [45]:
%%time

dist, top_k = index.search(query_embedding_np, 5)  # search
top_k

CPU times: user 4.94 ms, sys: 0 ns, total: 4.94 ms
Wall time: 3.9 ms


array([[ 16069, 154036,  66896,  13600, 192765]])

Time taken is 6 ms, which is close to 6 times more than previous search. Let's take a look at the results.

First one is same, let's have a look at the second and third result for comparison.

In [46]:
docs_df.iloc[154036].abstract

'  The current leading paradigm for temporal information extraction from text\nconsists of three phases: (1) recognition of events and temporal expressions,\n(2) recognition of temporal relations among them, and (3) time-line\nconstruction from the temporal relations. In contrast to the first two phases,\nthe last phase, time-line construction, received little attention and is the\nfocus of this work. In this paper, we propose a new method to construct a\nlinear time-line from a set of (extracted) temporal relations. But more\nimportantly, we propose a novel paradigm in which we directly predict start and\nend-points for events from the text, constituting a time-line without going\nthrough the intermediate step of prediction of temporal relations as in earlier\nwork. Within this paradigm, we propose two models that predict in linear\ncomplexity, and a new training loss using TimeML-style annotations, yielding\npromising results.\n'

In [47]:
docs_df.iloc[66896].abstract

'  It is commonly acknowledged that temporal expression extractors are important\ncomponents of larger natural language processing systems like information\nretrieval and question answering systems. Extraction and normalization of\ntemporal expressions in Turkish has not been given attention so far except the\nextraction of some date and time expressions within the course of named entity\nrecognition. As TimeML is the current standard of temporal expression and event\nannotation in natural language texts, in this paper, we present an analysis of\ntemporal expressions in Turkish based on the related TimeML classification\n(i.e., date, time, duration, and set expressions). We have created a lexicon\nfor Turkish temporal expressions and devised considerably wide-coverage\npatterns using the lexical classes as the building blocks. We believe that the\nproposed patterns, together with convenient normalization rules, can be readily\nused by prospective temporal expression extraction tools fo

The match is better for the query as we increase the nprobe number but time increases. It's a tradeoff between performance and accuracy.

Let's see times for few more nprobe numbers

In [48]:
index.nprobe = 5

In [49]:
%%time

dist, top_k = index.search(query_embedding_np, 5)  # search
top_k

CPU times: user 776 µs, sys: 3 ms, total: 3.78 ms
Wall time: 2.34 ms


array([[ 16069, 154036,  66896,  13600, 192765]])

In [50]:
index.nprobe = 100

In [51]:
%%time

dist, top_k = index.search(query_embedding_np, 5)  # search
top_k

CPU times: user 33.6 ms, sys: 1.99 ms, total: 35.6 ms
Wall time: 33.9 ms


array([[ 16069, 154036,  66896,  13600, 192765]])

As we can clearly see, increasing the nprobe number increases the number of cells that needs to be searched for the result to be returned. This increases the accuracy as set of data that is searched increases, but takes more time. Proper nprobe number can be selected based on requirements, as the time required to search in millions of documents can be significantly higher than the milliseconds we are seeing here for half a million docs.

FAISS also provides us with a lot of control compared to standar cosine_similarity that we used earlier with sentence-transformers.

#### Quantization

Another optimization technique is Product Quantization (PQ). 

Earlier, where we reduced the search time by reducing the search area. This helps in reducing search time. But what if, the dataset itself is huge and storing the complete vectors become an issue.

Quantization helps us reduce the size of vectors by approximating the similarity metric. It compresses the vectors that results in approximation of the similarity metric. Hence, it reduces the search time. It does this by following the steps - 

1. Split the original vector into sub-vectors.
2. Each set of subvectors, a clustering operation is performed which creates multiple centroids for each sub-vector set.
3. Then,in the original vector, each sub-vector is replaced by the ID of its nearest set-specific centroid.

IndexIVFPQ is used.

In [52]:
m = 8  # number of centroid IDs or basically number of sub-vectors in each compressed vector.
bits = 8 # size of each centroid
nlist = 200 # number of cells

quantizer = faiss.IndexFlatL2(dim) # same l2 dist flat index
index = faiss.IndexIVFPQ(quantizer, dim, nlist, m, bits) 

In [53]:
index.is_trained

False

In [54]:
%%time

index.train(abstract_embeddings_np)
index.is_trained

CPU times: user 6.39 s, sys: 157 ms, total: 6.55 s
Wall time: 6.56 s


True

In [55]:
index.add(abstract_embeddings_np)

In [56]:
index.nprobe

1

In [57]:
%%time

dist, top_k = index.search(query_embedding_np, 5)  # search
top_k

CPU times: user 676 µs, sys: 0 ns, total: 676 µs
Wall time: 827 µs


array([[ 16069, 245940, 368076, 273379, 483819]])

In [58]:
index.nprobe = 10

In [59]:
%%time

dist, top_k = index.search(query_embedding_np, 5)  # search
top_k

CPU times: user 938 µs, sys: 0 ns, total: 938 µs
Wall time: 1.03 ms


array([[ 16069,  13597, 192765,  66896, 204477]])

Further reduced the search time for nprob=10 from 6ms to 2ms. Results -

In [60]:
docs_df.iloc[13597].abstract

'  Automatic temporal ordering of events described in discourse has been of\ngreat interest in recent years. Event orderings are conveyed in text via va\nrious linguistic mechanisms including the use of expressions such as "before",\n"after" or "during" that explicitly assert a temporal relation -- temporal\nsignals. In this paper, we investigate the role of temporal signals in temporal\nrelation extraction and provide a quantitative analysis of these expres sions\nin the TimeBank annotated corpus.\n'

In [61]:
docs_df.iloc[192765].abstract

'  Automatic extraction of temporal information in text is an important\ncomponent of natural language understanding. It involves two basic tasks: (1)\nUnderstanding time expressions that are mentioned explicitly in text (e.g.,\nFebruary 27, 1998 or tomorrow), and (2) Understanding temporal information that\nis conveyed implicitly via relations. In this paper, we introduce CogCompTime,\na system that has these two important functionalities. It incorporates the most\nrecent progress, achieves state-of-the-art performance, and is publicly\navailable.1 We believe that this demo will be useful for multiple time-aware\napplications and provide valuable insight for future research in temporal\nunderstanding.\n'

The ordering of results has changes from our previous output even though the set of documents overlap. This is because of the approximation of the similarity metric.

In [62]:
index.make_direct_map()

In [63]:
index.reconstruct(16069).shape

(384,)

Referenced from - [FAISS exploration](https://www.pinecone.io/learn/faiss-tutorial/)